## import libraries and the data and check if there is any missing values in the dataset

In [334]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [335]:
insurance_df=pd.read_csv('insurance.csv')


In [336]:
print(insurance_df.isna().sum())

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [337]:
insurance_df.head(10)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


## we can encode the 2 columns sex and smoker to zeros and ones as they represent true and flase

In [338]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
se = LabelEncoder()
insurance_df ['sex']= le.fit_transform(insurance_df ['sex'])
insurance_df ['smoker']= se.fit_transform(insurance_df ['smoker'])


## and then we can split the fatures to x and y as y is the target index and the rest of features we can put them in x

In [339]:
x=insurance_df.iloc[:,:-1].values
y=insurance_df.iloc[:,-1].values

In [340]:
print (x)

[[19 0 27.9 0 1 'southwest']
 [18 1 33.77 1 0 'southeast']
 [28 1 33.0 3 0 'southeast']
 ...
 [18 0 36.85 0 0 'southeast']
 [21 0 25.8 0 0 'southwest']
 [61 0 29.07 0 1 'northwest']]


## Now it became so obvious for us that we need to encode the region so i used one hot encoder

In [341]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [5])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [342]:
print(x)

[[0.0 0.0 0.0 ... 27.9 0 1]
 [0.0 0.0 1.0 ... 33.77 1 0]
 [0.0 0.0 1.0 ... 33.0 3 0]
 ...
 [0.0 0.0 1.0 ... 36.85 0 0]
 [0.0 0.0 0.0 ... 25.8 0 0]
 [0.0 1.0 0.0 ... 29.07 0 1]]


## Now our data is ready for split it to train and test set

In [343]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state= 2022)

In [344]:
print(x_train)

[[0.0 1.0 0.0 ... 35.53 0 0]
 [0.0 0.0 0.0 ... 23.7 2 0]
 [0.0 0.0 1.0 ... 31.35 0 0]
 ...
 [0.0 0.0 0.0 ... 34.8 1 0]
 [0.0 0.0 1.0 ... 33.33 0 0]
 [0.0 0.0 1.0 ... 38.94 2 1]]


In [345]:
y_train=y_train.reshape(-1,1)

##After spliting the data we need to use feature scaling as some models need to do that before fitting it to the data as svr ,multilinear regression,etc





In [346]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
sc_y = StandardScaler()
x_train_sc = sc_x.fit_transform(x_train)
y_train_sc = sc_y.fit_transform(y_train)
x_test_sc=sc_x.transform(x_test)
y_test_sc=sc_y.transform(y_test.reshape(-1,1))


## Now our first model we can try multiple linear regression model

In [347]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train_sc, y_train_sc)
y_pred = regressor.predict(x_test_sc)

In [348]:
r2_score(y_test_sc, y_pred)

0.802631009020436

##  second model we can try support vector regression (SVR) model

In [349]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(x_train_sc, y_train_sc)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR()

In [350]:
y_pred=sc_y.inverse_transform(regressor.predict(sc_x.transform(x_test)).reshape(-1,1))
r2_score(y_test, y_pred)

0.8842243109114281

## Third we can use the polynomial regression model

In [351]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree = 3)
X_poly = poly_reg.fit_transform(x_train)
regressor = LinearRegression()
regressor.fit(X_poly, y_train)
y_pred = regressor.predict(poly_reg.transform(x_test))

In [352]:
r2_score(y_test, y_pred)

0.8766231653502087

##Fourth lets try Decision Tree Regression model

In [353]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

In [354]:
r2_score(y_test, y_pred)

0.7146755740982859

## 5th-let's try random forest regreesion

In [355]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 30, random_state = 0)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

<ipython-input-355-a3ab72e23559>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(x_train, y_train)


In [356]:
r2_score(y_test, y_pred)

0.8666590720696801

##6th and last one bagging regression

In [357]:
from sklearn.ensemble import BaggingRegressor
bagging_regressor = BaggingRegressor(n_estimators=10)

bagging_regressor.fit(x_train, y_train)

y_pred = bagging_regressor.predict(x_test)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


In [358]:
r2_score(y_test, y_pred)

0.8343315438758864

###  **Conclusion**
we got a good performance in most of the model but the best model that fit our data was SVR so why SVR was the best for our data and when to use it


1. SVR can handle complex relationships between input features and the target variable by using kernel functions to transform the data into a higher-dimensional space.
2. It is robust to outliers, thanks to epsilon-insensitive loss functions that tolerate a certain degree of error, preventing outliers from dominating the training process.
3. SVR offers control over model complexity through parameters like C (regularization parameter) and the choice of kernel function, allowing a balance between fitting the data well and maintaining good generalization performance.
4. It supports various kernel functions (e.g., linear, polynomial, radial basis function) that can be chosen based on the data characteristics and underlying relationships between variables.
5. SVR's versatility and adaptability make it suitable for handling different types of data and capturing various patterns effectively.



## > **In summary**
 SVR is a regression algorithm that can handle complex relationships, is robust to outliers, provides control over model complexity, and offers flexibility with kernel functions for different data types.
